In [28]:
#get daily covid cases

import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import datetime
coviddata=pd.read_hdf('./covidtata.h5',key='updatefrom1130')

coviddata['new_cases_daily']=coviddata['新增无症状']+coviddata['新增本土确诊']
df_covid=coviddata[['省份', '城市', '日期','new_cases_daily']]

s1=df_covid['日期']>"2022-02-01" #2022年2月1号开始区分 新增本土无症状（即表里的新增无症状）和新增本土确诊
s2=df_covid['省份'].isin(['北京','上海','天津','重庆'])
s3=~df_covid['城市'].isin(['北京','上海','天津','重庆'])

df_covid['城市']=df_covid['城市'].str.replace('伊犁州','伊犁')

df_covid=df_covid[s1] #去掉2022年前的
df_covid=df_covid[~(s2&s3)] #去掉直辖市分区的统计，因为后面没抓到其实，而且分析也没有意义

df_covid['日期'] = pd.to_datetime(df_covid['日期']).dt.date

def getyr(d):
    return str(d.isocalendar()[0])

df_covid['yr']=df_covid['日期'].apply(getyr)


#rolling average
df_covid=df_covid.sort_values(by='日期',ascending=True).reset_index(drop=True)
df_covid['cases_last7d_max'] = df_covid.groupby('城市')['new_cases_daily'].transform(lambda x: x.rolling(7, 1).max())

df_covid=df_covid.rename(columns={'日期':'date'})
df_covid.head(5)


,省份,城市,date,new_cases_daily,yr,cases_last7d_max
0,台湾,台湾,2022-02-02,0,2022,0.0
1,广西,梧州,2022-02-02,0,2022,0.0
2,湖南,郴州,2022-02-02,0,2022,0.0
3,山东,济宁,2022-02-02,0,2022,0.0
4,浙江,舟山,2022-02-02,0,2022,0.0


In [2]:
coviddata[coviddata['城市']=='上海']

,省份,城市,日期,累计确诊,新增确诊,累计治愈,新增治愈,累计死亡,新增死亡,新增无症状,新增本土确诊,风险地区,new_cases_daily
202479,上海,上海,2022-11-28,64530.0,26.0,63776.0,10.0,595.0,0.0,158,20,NaN,178
202480,上海,上海,2022-11-27,64504.0,20.0,63766.0,10.0,595.0,0.0,128,16,NaN,144
202481,上海,上海,2022-11-26,64484.0,19.0,63756.0,4.0,595.0,0.0,119,11,NaN,130
202482,上海,上海,2022-11-25,64465.0,14.0,63752.0,4.0,595.0,0.0,78,11,NaN,89
202483,上海,上海,2022-11-24,64451.0,18.0,63748.0,3.0,595.0,0.0,77,9,NaN,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...
203521,上海,上海,2020-01-21,9.0,8.0,0.0,0.0,0.0,0.0,0,0,NaN,0
203522,上海,上海,2020-01-20,1.0,1.0,0.0,0.0,0.0,0.0,0,0,NaN,0
376976,上海,上海,2022-11-29,NaN,NaN,NaN,NaN,NaN,NaN,176,11,122.0,187
377274,上海,上海,2022-11-30,NaN,NaN,NaN,NaN,NaN,NaN,174,23,138.0,197


In [17]:
pd.read_excel('./Amap Conjestion.xlsx').to_hdf('./amap.h5',key='raw1129')

In [30]:
# get 7d rolling amap traffic

amap=pd.read_hdf('./amap.h5',key='raw1129')
amap=pd.melt(amap,id_vars=['城市','省','城市线'],var_name='date', value_name='traffic')
amap['date']=pd.to_datetime(amap['date']).dt.date


def getyr(d):
    return str(d.isocalendar()[0])
def getdt(d):
    return str(d)[-5:]

amap['yr']=amap['date'].apply(getyr)
amap['dt']=amap['date'].apply(getdt)

amap=amap.sort_values(by='date',ascending=True).reset_index(drop=True)

amap_2019=amap[amap['yr'].str.contains('2019')]
amap_2019['traffic_last7d_mean'] = amap_2019.groupby('城市')['traffic'].transform(lambda x: x.rolling(30, 1).mean())
amap_2019=amap_2019[['省','城市','dt','traffic_last7d_mean']]
amap_2019=amap_2019.rename(columns={"traffic_last7d_mean":"traffic_19_rolling_mean"})

amap_2022=amap[amap['yr'].str.contains('2022')]
amap_2022=amap_2022.merge(amap_2019,how='left',on=['省','城市','dt'])
amap_2022['traffic_last7d_mean'] = amap_2022.groupby('城市')['traffic'].transform(lambda x: x.rolling(7, 7).mean())
amap_2022['traffic_status']=amap_2022['traffic_last7d_mean']/amap_2022['traffic_19_rolling_mean']-1

amap_2022['traffic_change']=amap_2022['traffic_last7d_mean']-amap_2022.groupby('城市')['traffic_last7d_mean'].transform(lambda x: x.shift(periods=7))

amap_2022.tail(5)

,城市,省,城市线,date,traffic,yr,dt,traffic_19_rolling_mean,traffic_last7d_mean,traffic_status,traffic_change
33295,青岛,山东,二线,2022-12-01,1.67,2022,12-01,1.709667,1.652857,-0.033228,-7.857143e-02
33296,苏州,江苏,二线,2022-12-01,1.58,2022,12-01,1.493667,1.572857,0.053018,4.571429e-02
33297,杭州,浙江,二线,2022-12-01,1.73,2022,12-01,1.526000,1.557143,0.020408,5.857143e-02
33298,潍坊,山东,二线,2022-12-01,1.35,2022,12-01,1.583667,1.295714,-0.181826,-2.057143e-01
33299,肇庆,广东,三线及以下,2022-12-01,1.36,2022,12-01,1.435000,1.432857,-0.001493,2.220446e-16


In [26]:
#疫情新增 by 人口 分析 

import numpy as np

city_popu=pd.read_hdf('./citypopu.h5',key='raw2')
city_popu['城市']=city_popu['城市'].str.replace('市',"")
city_popu['常住人口']=pd.to_numeric(city_popu['常住人口'])

dfo=df_covid.merge(city_popu,on='城市',how='inner')


def catgorize(num):
    num=float(num)
    level=np.nan
    
    if num ==0:
        level= '0 无疫情区域'
    if 0 < num <=10:
        level ="1 轻微疫情区域"
    if num > 10 and num <=100:
        level ="2 较严重疫情区域"
    if num > 100:
        level ="3 严重疫情区域"

    return level


dfo['covid_level']=dfo['cases_last7d_max'].apply(catgorize)

show=dfo.copy()
c1=show['covid_level']==np.nan
show=show[~c1]

pv_value=pd.pivot_table(show, values='常住人口', index=['date'],columns=['covid_level'], aggfunc=np.sum)
pv_value=pv_value.div(pv_value.sum(axis=1), axis=0)
pv_value['sum']=1
pv_value=pv_value[['sum','0 无疫情区域', '1 轻微疫情区域', '2 较严重疫情区域', '3 严重疫情区域']]
pv_value.tail(22).to_clipboard()
pv_value.tail(5)

covid_level,sum,0 无疫情区域,1 轻微疫情区域,2 较严重疫情区域,3 严重疫情区域
date,,,,,
2022-11-27,1,0.019003,0.111150,0.480770,0.389077
2022-11-28,1,0.017357,0.099364,0.483812,0.399468
2022-11-29,1,0.014529,0.089432,0.462864,0.433175
2022-11-30,1,0.015204,0.080407,0.466312,0.438077
2022-12-01,1,0.022072,0.068784,0.471009,0.438136


In [27]:
#疫情 新增by 人口，典型城市
show['date']=show['date'].astype('string') 
l=show[show['date']=='2022-11-30']
l=l.sort_values(by=['covid_level','常住人口'],ascending=False)
table=l.groupby('covid_level').head(5)[['covid_level','date','省份', '城市','new_cases_daily','cases_last7d_max', '常住人口']].sort_values(by=['covid_level','cases_last7d_max','常住人口'],ascending=[False,False,False])
table.to_clipboard()
table.head(5)

,covid_level,date,省份,城市,new_cases_daily,cases_last7d_max,常住人口
81846,3 严重疫情区域,2022-11-30,重庆,重庆,6639,9685.0,2542
7254,3 严重疫情区域,2022-11-30,广东,广州,6942,8443.0,1874
23868,3 严重疫情区域,2022-11-30,北京,北京,5043,5043.0,2189
24474,3 严重疫情区域,2022-11-30,广东,深圳,228,228.0,1763
61281,3 严重疫情区域,2022-11-30,上海,上海,197,197.0,2488


In [32]:
#traffic status by covid level
dfo=amap_2022.merge(show,on=['城市','yr','date'],how='inner')
result=pd.pivot_table(dfo, values='traffic_status', index=['date'],columns=['covid_level'], aggfunc=np.mean)
result.to_clipboard()
result.head(10)

covid_level
date


In [121]:
# dfo[dfo['date']==d2022-11-30']
dfo['date']=dfo['date'].astype('string') 
dfo['covid_rank']=dfo.groupby(by=['date'])['cases_last7d_sum'].rank(method="dense", ascending=True)
dfo[dfo['date']=='2022-11-30'][['城市','date','covid_level','cases_last7d_sum','traffic_change','traffic_status','covid_rank']].to_clipboard()
dfo.columns

Index(['城市', '省', '城市线', 'date', 'traffic', 'yr', 'dt',
       'traffic_19_rolling_mean', 'traffic_last7d_mean', 'traffic_status',
       'traffic_change', '省份', 'new_cases_daily', 'cases_last7d_max',
       'cases_last7d_sum', 'cases_14_7d_sum', 'case_growth', 'covid_growth',
       'case_growth_abs', '常住人口', 'covid_level', 'covid_rank'],
      dtype='object')

In [37]:
# 疫情增速分析

#rolling average
df_covid=df_covid.sort_values(by='date',ascending=True).reset_index(drop=True)
df_covid['cases_last7d_sum'] = df_covid.groupby('城市')['new_cases_daily'].transform(lambda x: x.rolling(7, 7).sum())
df_covid['cases_14_7d_sum'] = df_covid.groupby('城市')['new_cases_daily'].transform(lambda x: x.rolling(14, 14).sum()) - df_covid.groupby('城市')['new_cases_daily'].transform(lambda x: x.rolling(7, 7).sum())
df_covid['case_growth']=df_covid['cases_last7d_sum']/df_covid['cases_14_7d_sum'] -1


def catgorize2(num,g):
    num=float(num)
    level='0 无疫情或轻微疫情'
    if num <=50:
        level= '0 无疫情或轻微疫情'
    else:
        if g<=-0.5:
            level ='1 快速降低'
        elif -0.5<g<=-0.1:
            level ='2 缓慢降低'
        elif -0.1<g<=0.1:
            level ='3 维持'
        elif 0.1<g<=1:
            level ='4 缓慢增长'
        elif g>1:
            level ='5 快速增长'
    return level

df_covid['covid_growth']=df_covid.apply(lambda x:catgorize2(x['cases_last7d_sum'],x['case_growth']),axis=1)
# df_covid[df_covid['城市'].str.contains('广州')].to_clipboard()

city_popu=pd.read_hdf('./citypopu.h5',key='raw2')
city_popu['城市']=city_popu['城市'].str.replace('市',"")
city_popu['常住人口']=pd.to_numeric(city_popu['常住人口'])

df_growth=df_covid.merge(city_popu,on='城市',how='inner')

pd.pivot_table(df_growth, values='常住人口', index=['date'],columns=['covid_growth'], aggfunc=np.sum).to_clipboard()


In [54]:
city_popu=pd.read_hdf('./citypopu.h5',key='raw2')
city_popu['城市']=city_popu['城市'].str.replace('市',"")
city_popu['常住人口']=pd.to_numeric(city_popu['常住人口'])

df_amap=amap_2022.merge(city_popu,on='城市',how='inner')

df_amap['traffic_change_last7d'] = df_amap.groupby('城市')['traffic_status'].transform(lambda x: x.rolling(7, 1).mean())

def catgorize2(g):
    level='5 社会面流动性无影响'
    g=float(g)
    if g <=-0.25:
        level= '0 社会面基本停滞'
    elif -0.25<g<=-0.15:
        level ='1 社会面流动性显著降低'
    elif -0.15<g<=-0.05:
        level ='4 社会面流动性轻微降低'
    elif g>-0.05:
        level ='5 社会面流动性无影响'
    return level

df_amap['traffic_change_level']=df_amap['traffic_change_last7d'].apply(catgorize2)


pv_value=pd.pivot_table(df_amap, values='常住人口', index=['date'],columns=["traffic_change_level"], aggfunc=np.sum)
pv_value=pv_value.div(pv_value.sum(axis=1), axis=0)
pv_value['sum']=1
pv_value=pv_value[['sum','0 社会面基本停滞', '1 社会面流动性显著降低', '4 社会面流动性轻微降低', '5 社会面流动性无影响']]
pv_value=pv_value.fillna(0)
pv_value.tail(280).to_clipboard()
pv_value.tail(280)

traffic_change_level,sum,0 社会面基本停滞,1 社会面流动性显著降低,4 社会面流动性轻微降低,5 社会面流动性无影响
date,,,,,
2022-02-25,1,0.000000,0.010354,0.007920,0.981726
2022-02-26,1,0.000000,0.010354,0.007920,0.981726
2022-02-27,1,0.000000,0.013695,0.004579,0.981726
2022-02-28,1,0.000000,0.013695,0.004579,0.981726
2022-03-01,1,0.000000,0.013695,0.004579,0.981726
...,...,...,...,...,...
2022-11-27,1,0.124082,0.142934,0.164240,0.568744
2022-11-28,1,0.135962,0.159269,0.213452,0.491317
2022-11-29,1,0.135962,0.191341,0.183524,0.489172


In [63]:
df_amap[df_amap['traffic_change_level']=='0 社会面基本停滞'].to_clipboard()